In [ ]:
import json
import subprocess
from pathlib import Path

## 1. Baseline Evaluation

Rule-based classifier on automotive test set.

In [ ]:
# Run baseline evaluation
result = subprocess.run(
    ["python", "scripts/evaluate_domain.py", "--test", "data/automotive_domain_test.jsonl"],
    capture_output=True,
    text=True,
    env={"PYTHONPATH": "."},
)
baseline = json.loads(result.stdout.split('\n')[0])
print(json.dumps(baseline, indent=2))

## 2. QLoRA Fine-Tuning

Train LoRA adapters on automotive domain data.

In [ ]:
# Train QLoRA (requires GPU + requirements-train.txt)
# !python scripts/train_qlora.py --model microsoft/phi-2 --epochs 3 --output checkpoints/automotive-adapter

## 3. AWQ Export

Export quantized model for edge deployment.

In [ ]:
# Export AWQ (requires requirements-awq.txt)
# !python scripts/export_awq.py --model microsoft/phi-2 --adapter checkpoints/automotive-adapter --output exports/automotive-awq-4bit

## 4. Comparison Table

| Configuration | Accuracy | Latency (ms) | Memory (GB) | Target Met |
|--------------|----------|--------------|-------------|------------|
| Baseline (rule) | 93.3% | 0.01 | <0.1 | ✓ Latency |
| + QLoRA | TBD | TBD | ~2.5 | TBD |
| + AWQ 4-bit | TBD | TBD | <2.0 | TBD |